In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=41b57d05c9183a1eb715fe8a67c4b9e912cdd1ecb7

In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [12]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data/")
data = loader.load()

In [13]:
print(data)

[Document(page_content='1\nFoundations & Trends in Multimodal Machine Learning:\nPrinciples, Challenges, and Open Questions\nPAUL PU LIANG, AMIR ZADEH, and LOUIS-PHILIPPE MORENCY,\nMachine Learning Department and Language Technologies Institute, Carnegie Mellon University, USA\nMultimodal machine learning is a vibrant multi-disciplinary research field that aims to design computer agents\nwith intelligent capabilities such as understanding, reasoning, and learning through integrating multiple\ncommunicative modalities, including linguistic, acoustic, visual, tactile, and physiological messages. With\nthe recent interest in video understanding, embodied autonomous agents, text-to-image generation, and\nmultisensor fusion in application domains such as healthcare and robotics, multimodal machine learning\nhas brought unique computational and theoretical challenges to the machine learning community given the\nheterogeneity of data sources and the interconnections often found between modali

In [14]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [15]:
len(text_chunks)

36

In [16]:
#get the third chunk
text_chunks[2]

Document(page_content='Foundations & Trends in Multimodal Machine Learning 1:3\n(2)Alignment (§4): How can we identify the connections and interactions between modality\nelements? Alignment is challenging since it may depend on long-range dependencies, involves\nambiguous segmentation (e.g., words or utterances), and could be either one-to-one, many-\nto-many, or not exist at all. We cover (1) discrete alignment : identifying connections between\ndiscrete elements across modalities, (2) continuous alignment : modeling alignment between con-\ntinuous modality signals with ambiguous segmentation, and (3) contextualized representations :\nlearning better representations by capturing cross-modal interactions between elements.\n(3)Reasoning (§5) is defined as composing knowledge, usually through multiple inferential steps,\nthat exploits the problem structure for a specific task. Reasoning involves (1) modeling the\nstructure over which composition occurs, (2) the intermediate concepts in t

In [17]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [18]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [22]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF", filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf")

'/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/45167a542b6fa64a14aea61a4c468bbbf9f258a8/mistral-7b-instruct-v0.1.Q4_K_M.gguf'

In [23]:
!ls

drive  sample_data


In [24]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/45167a542b6fa64a14aea61a4c468bbbf9f258a8/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [25]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [28]:
# query = "What is linear regression model"
query = "What is the difference between CLIP and FLamingo?"

In [29]:
qa.run(query)

Llama.generate: prefix-match hit


' CLIP (Concept Labelled Image Priors) and FLamingo are both large language models, but they differ in the tasks they were trained on and the way they interact with their corresponding image embeddings. CLIP was primarily trained on image captioning and task-specific labels, while FLamingo was primarily trained on generative modeling of images and can be used for tasks such as style transfer and denoising. In terms of interaction with image embeddings, CLIP generates text descriptions of images, while FLamingo allows users to generate new images by interacting with the pre-trained language model.\nReference(s):\nhttps://arxiv.org/abs/2103.04566\nhttps://arxiv.org/abs/2012.07897'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")